### 训练词向量

In [2]:
import pandas as pd

In [3]:
csv_ptah = './new_data/sqlResult_1558435.csv'

In [4]:
data = pd.read_csv(csv_ptah, encoding='gb18030')

In [5]:
data = data.fillna('') #将nan值转换为0
data.head()

,id,author,source,content,feature,title,url
0,89617,,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [6]:
contents = data['content'].tolist() # 转化为列表
contents[0:5]

['此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n',
 '骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考虑性能而去屏蔽掉小核心。相反，他们正联手微软，找到一种适合桌面平台的、兼顾性能和功耗的完美方案。\r\n报道称，微软已经拿到了一些新的源码，以便Windows 10更好地理解big.little架构。\r\n资料显示，骁龙835作为一款集成了CPU、GPU、基带、蓝牙/Wi-Fi的SoC，比传统的Wintel方案可以节省至少30%的PCB空间。\r\n按计划，今年Q4，华硕、惠普、联想将首发骁龙835 Win10电脑，预计均是二合一形态的产品。\r\n当然，高通骁龙只是个开始，未来也许还能见到三星Exynos、联发科、华为麒麟、小米澎湃等进入Windows 10桌面平台。\r\n',
 '此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n至于电池缩水，可能与刘作虎所说，一加手机5要做市面最轻薄大屏旗舰的设定有关。\r\n按照目前掌握的资料，一加手机5拥有5.5寸1080P三星AMOLED显示屏、6G/8GB RAM，64GB/128GB ROM，双1600万摄像头，备货量“惊喜”。\r\n根据京东泄露的信息，一加5起售价是xx99元，应该是在2799/2899/2999中的某个。\r\n',
 '这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n',
 '（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清晨交警发现有一女子赤裸上身，行走在南坪快速上，期间还起了轻生年头，一辅警发现后赶紧为其披上黄衣，并一路劝说她。\r\n那么事发时\r\n到底都发生了些什么呢？\r\n南都记者带您一起还原现场\r\n南都记者在龙岗大队坂田中队见到了辅警刘青（发现女生的辅警），一位外表高大帅气，说

In [7]:
import jieba
def cut(string):
    """切词"""
    return ' '.join(jieba.cut(string))

In [8]:
import re
def token(string):
    """匹配文字"""
    return re.findall(r'[\d\w]+', string)

In [9]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(lineno)d -  %(message)s')
logger = logging.getLogger(__name__)

def data_processing(contents):
    with open('news.txt', 'w') as f:
        for content in contents:
            content = ' '.join(token(content))
            content = cut(content)
            f.write(content + '\n')
data_processing(contents)

Building prefix dict from the default dictionary ...
2019-08-21 16:33:54,548 - DEBUG - 111 -  Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\huangm\AppData\Local\Temp\jieba.cache
2019-08-21 16:33:54,563 - DEBUG - 131 -  Loading model from cache C:\Users\huangm\AppData\Local\Temp\jieba.cache
Loading model cost 0.642 seconds.
2019-08-21 16:33:55,203 - DEBUG - 163 -  Loading model cost 0.642 seconds.
Prefix dict has been built succesfully.
2019-08-21 16:33:55,204 - DEBUG - 164 -  Prefix dict has been built succesfully.


In [12]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
model = Word2Vec(LineSentence('news.txt'), size=35, workers=4)
model.most_similar('江西', topn=20)

2019-08-21 16:41:03,049 - WARNING - 723 -  consider setting layer size to a multiple of 4 for greater performance
2019-08-21 16:41:03,050 - INFO - 1588 -  collecting all words and their counts
2019-08-21 16:41:03,051 - INFO - 1573 -  PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-08-21 16:41:04,230 - INFO - 1573 -  PROGRESS: at sentence #10000, processed 4177862 words, keeping 147449 word types
2019-08-21 16:41:04,677 - INFO - 1573 -  PROGRESS: at sentence #20000, processed 5817086 words, keeping 165696 word types
2019-08-21 16:41:05,144 - INFO - 1573 -  PROGRESS: at sentence #30000, processed 7473755 words, keeping 179800 word types
2019-08-21 16:41:05,705 - INFO - 1573 -  PROGRESS: at sentence #40000, processed 9260675 words, keeping 194376 word types
2019-08-21 16:41:06,315 - INFO - 1573 -  PROGRESS: at sentence #50000, processed 11159124 words, keeping 208757 word types
2019-08-21 16:41:06,781 - INFO - 1573 -  PROGRESS: at sentence #60000, processed 12960711

2019-08-21 16:41:50,428 - INFO - 1305 -  EPOCH 3 - PROGRESS: at 69.54% examples, 1027849 words/s, in_qsize 5, out_qsize 0
2019-08-21 16:41:51,429 - INFO - 1305 -  EPOCH 3 - PROGRESS: at 77.49% examples, 1033083 words/s, in_qsize 2, out_qsize 0
2019-08-21 16:41:52,433 - INFO - 1305 -  EPOCH 3 - PROGRESS: at 85.73% examples, 1038858 words/s, in_qsize 1, out_qsize 0
2019-08-21 16:41:53,434 - INFO - 1305 -  EPOCH 3 - PROGRESS: at 94.19% examples, 1044098 words/s, in_qsize 0, out_qsize 0
2019-08-21 16:41:54,433 - INFO - 349 -  worker thread finished; awaiting finish of 3 more threads
2019-08-21 16:41:54,435 - INFO - 1305 -  EPOCH 3 - PROGRESS: at 99.81% examples, 1026780 words/s, in_qsize 2, out_qsize 1
2019-08-21 16:41:54,436 - INFO - 349 -  worker thread finished; awaiting finish of 2 more threads
2019-08-21 16:41:54,449 - INFO - 349 -  worker thread finished; awaiting finish of 1 more threads
2019-08-21 16:41:54,451 - INFO - 349 -  worker thread finished; awaiting finish of 0 more thread

[('安徽', 0.8971200585365295),
 ('山西', 0.8944082856178284),
 ('湖南', 0.8936129212379456),
 ('湖北', 0.8843142986297607),
 ('陕西', 0.8833469152450562),
 ('广西', 0.8700163960456848),
 ('云南', 0.8661714196205139),
 ('山东', 0.8388182520866394),
 ('南昌', 0.8302770256996155),
 ('河北', 0.8255032896995544),
 ('福建', 0.8239394426345825),
 ('宁夏', 0.8208080530166626),
 ('浙江', 0.8180115222930908),
 ('黑龙江', 0.8155186176300049),
 ('内蒙古', 0.7982271313667297),
 ('河南', 0.7980940341949463),
 ('甘肃', 0.7975690364837646),
 ('云南省', 0.790233850479126),
 ('江西省', 0.7888780832290649),
 ('贵州', 0.7877655625343323)]

### 获取最相近的词

In [13]:
model.most_similar('说', topn=20)

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('表示', 0.9060478806495667),
 ('指出', 0.8532556295394897),
 ('认为', 0.8503166437149048),
 ('看来', 0.8210107088088989),
 ('坦言', 0.8129802942276001),
 ('告诉', 0.7829445600509644),
 ('介绍', 0.7655443549156189),
 ('明说', 0.7475131750106812),
 ('称', 0.7271586060523987),
 ('透露', 0.71549391746521),
 ('强调', 0.714995265007019),
 ('所说', 0.7060800790786743),
 ('中说', 0.691342830657959),
 ('文说', 0.690251350402832),
 ('深有体会', 0.6787254810333252),
 ('时说', 0.6612800359725952),
 ('特别强调', 0.6533271670341492),
 ('称赞', 0.6101324558258057),
 ('确信', 0.6040674448013306),
 ('提到', 0.602391242980957)]

In [21]:
from collections import defaultdict

def get_similar_words(init_words, model):
    unseen = init_words
    seen = defaultdict(int)
    max_size = 200
    
    while unseen and len(seen) < max_size:
        if len(seen) % 50 == 0:
            print('seen length:{}'.format(len(seen)))
        node = unseen.pop(0)
        
        new_nodes = [w for w,s in model.most_similar(node, topn=20)]
        
        unseen += new_nodes
        
        seen[node] += 1
    return seen

similar_words = get_similar_words(['说'], model)
similar_words = sorted(similar_words.items(), key = lambda x:x[1], reverse=True)
similar_words

seen length:0


F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]


seen length:50
seen length:100
seen length:150


[('说', 28),
 ('表示', 25),
 ('指出', 25),
 ('坦言', 24),
 ('认为', 23),
 ('透露', 22),
 ('所说', 17),
 ('看来', 16),
 ('告诉', 16),
 ('明说', 16),
 ('称', 15),
 ('特别强调', 14),
 ('提到', 14),
 ('中说', 11),
 ('深有体会', 11),
 ('介绍', 10),
 ('强调', 9),
 ('时说', 9),
 ('相信', 8),
 ('确信', 7),
 ('建议', 7),
 ('普遍认为', 7),
 ('提及', 7),
 ('文说', 6),
 ('直言', 6),
 ('中称', 6),
 ('写道', 6),
 ('呼吁', 5),
 ('说道', 5),
 ('表明', 4),
 ('而言', 4),
 ('写信给', 4),
 ('坦承', 4),
 ('问', 4),
 ('供图', 4),
 ('还称', 4),
 ('说法', 4),
 ('引用', 4),
 ('上称', 4),
 ('原话', 4),
 ('时称', 4),
 ('接受', 3),
 ('看法', 3),
 ('眼中', 3),
 ('地说', 3),
 ('常务', 3),
 ('副', 3),
 ('黄进', 3),
 ('列举', 3),
 ('反复强调', 3),
 ('如是说', 3),
 ('称赞', 2),
 ('嚷嚷', 2),
 ('对此', 2),
 ('事实上', 2),
 ('来说', 2),
 ('肯定', 2),
 ('觉得', 2),
 ('讨薪', 2),
 ('谈到', 2),
 ('留意到', 2),
 ('看到', 2),
 ('邓摄', 2),
 ('见到', 2),
 ('李骥志摄', 2),
 ('童岚', 2),
 ('了解', 2),
 ('获悉', 2),
 ('援外', 2),
 ('主任', 2),
 ('尼格', 2),
 ('研修班', 2),
 ('办公室', 2),
 ('法学所', 2),
 ('农艺师', 2),
 ('郭', 2),
 ('宣称', 2),
 ('假新闻', 2),
 ('居心', 2),
 ('声称', 2),
 ('五角大楼', 

### TFIDF Key words

In [22]:
contents[0]

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n'

In [23]:
def document_frequency(word):
    """计算所有文本包含词的文本数"""
    return sum(1 if word in content else 0 for content in contents)

In [40]:
import math
def idf(word):
    """计算逆文本频率idf"""
    return math.log10(len(contents) / (document_frequency(word) + 1))

In [41]:
def tf(word, document):
    """词频tf"""
    words = document.split() 
    return sum(1 if word == w else 0 for w in words)

In [42]:
def tfidf(document):
    """获得一个文本所有词的tfidf"""
    words = set(document.split()) #去重
    
    tf_idf = [(w, tf(w, document) * idf(w)) for w in words]
    
    return sorted(tf_idf, key=lambda x:x[1], reverse=True)

In [43]:
document = cut(''.join(token(contents[5])))
tfidf(document)

[('板块', 21.684577445197387),
 ('A股', 18.543072429053936),
 ('上涨', 17.34088470221026),
 ('金融股', 14.635277292638518),
 ('MSCI', 13.497541464062904),
 ('指数', 12.43997666414595),
 ('格局', 11.552518175000666),
 ('市场', 10.675659598434926),
 ('涨幅', 9.976165630578302),
 ('纳入', 9.389553024189413),
 ('成分股', 8.97995979441414),
 ('权重', 8.127969825318537),
 ('证券', 7.700046864070869),
 ('存量', 7.3704509587380755),
 ('利好', 7.274194268763405),
 ('金融', 7.224551314452519),
 ('资金', 7.112730635739216),
 ('震荡', 6.909079395240995),
 ('非银行', 6.658224066789146),
 ('收红', 6.560526931713512),
 ('风格', 6.219796332877202),
 ('白马股', 6.214526567556433),
 ('信托', 5.717879277260477),
 ('交替', 5.663574785173248),
 ('行业', 5.657665458003947),
 ('222', 5.637644830844512),
 ('周四', 5.637644830844512),
 ('周三', 5.379820468124088),
 ('博弈', 5.329119187724495),
 ('市值', 5.122852433378189),
 ('增量', 5.119328741065615),
 ('占', 5.052920506231903),
 ('两市', 5.001149776536367),
 ('提振对', 4.9523613237924735),
 ('MSCIEM', 4.9523613237924735),
 

### TFIDF Vectorizezd

In [44]:
# np.where(condition, x, y) 满足条件(condition)，输出x，不满足输出y ,无x,y, 输入非零值

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorized = TfidfVectorizer(max_features=10000)
docs = [cut(''.join(token(content))) for content in contents[:50]]
tf_idf = vectorized.fit_transform(docs)
tf_idf.shape

(50, 5877)

In [54]:
#vectorized.vocabulary_ # A mapping of terms to feature indices.

In [65]:
# np.where(tf_idf[0].toarray())
# tf_idf[0].toarray().shape #(1, 5877) 二维
# tf_idf[0].toarray()[0].shape #(5877,) 一维

(1, 5877)

In [63]:
# scipy.spatial.distance.cosine(u，v)
# 作用：计算一维数组之间的余弦距离。（注意是一维数组的余弦距离）

In [68]:
from scipy.spatial.distance import cosine

def distance(v1, v2):
    """计算两个一维数组的余弦值"""
    return cosine(v1, v2)

v1  = tf_idf[2].toarray()[0]
v2 = tf_idf[0].toarray()[0]

In [71]:
doc1 = docs[2]
doc2 = docs[0]
doc1, doc2

('此前 的 一加 3T 搭载 的 是 3400mAh 电池 DashCharge 快充 规格 为 5V4A 至于 电池 缩水 可能 与 刘作 虎 所说 一加 手机 5 要 做 市面 最 轻薄 大屏 旗舰 的 设定 有关 按照 目前 掌握 的 资料 一加 手机 5 拥有 55 寸 1080P 三星 AMOLED 显示屏 6G8GBRAM64GB128GBROM 双 1600 万 摄像头 备货 量 惊喜 根据 京东 泄露 的 信息 一加 5 起 售价 是 xx99 元 应该 是 在 279928992999 中 的 某个',
 '此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息')

In [70]:
distance(v1, v2)

0.9727365601975771

### Build Search Engine

```
Input: Words
Output: Documents
```

In [75]:
def naive_search(keywords):
    news_ids = [i for i, n in enumerate(contents) if all(w in n for w in keywords)]
    # O(D * w)
    return news_ids

In [76]:
naive_search('美军 司令 航母'.split())

[2844,
 21484,
 21523,
 21784,
 21901,
 22987,
 33522,
 33770,
 34532,
 34887,
 37846,
 38049,
 38096,
 45474,
 45507,
 87483,
 88997]

In [77]:
contents = [' '.join(token(content)) for content in contents]
contents = [cut(content) for content in contents]

In [78]:
tf_idf.shape

(50, 5877)

In [81]:
word_2_id = vectorized.vocabulary_

In [87]:
# tf.transpose： 转置
transposed_x = tf_idf.transpose().toarray()

In [82]:
word_2_id['今天']

982

In [83]:
id_2_word = {d: w for w, d in word_2_id.items()}
id_2_word

{3854: '此外',
 3683: '本周',
 34: '12',
 3502: '日起',
 2563: '小米',
 3058: '手机',
 49: '15',
 3705: '机型',
 1363: '其余',
 3576: '暂停',
 3602: '更新',
 1809: '发布',
 2772: '开发',
 1135: '体验版',
 1385: '内测',
 4489: '稳定版',
 3575: '暂不受',
 2858: '影响',
 4430: '确保',
 2635: '工程师',
 1869: '可以',
 5654: '集中',
 1310: '全部',
 4579: '精力',
 5387: '进行',
 4589: '系统优化',
 2629: '工作',
 3640: '有人',
 4189: '猜测',
 810: '主要',
 4251: '用到',
 327: 'miui9',
 4416: '研发',
 823: '之中',
 326: 'miui8',
 1771: '去年',
 5242: '距今已有',
 436: '一年',
 3641: '有余',
 3516: '时候',
 3603: '更新换代',
 2847: '当然',
 1348: '关于',
 4432: '确切',
 1199: '信息',
 3025: '我们',
 5344: '还是',
 4550: '等待',
 2456: '官方消息',
 5799: '骁龙',
 240: '835',
 1144: '作为',
 2017: '唯一',
 5447: '通过',
 379: 'windows10',
 3792: '桌面',
 2694: '平台',
 5047: '认证',
 268: 'arm',
 2210: '处理器',
 5841: '高通',
 2833: '强调',
 611: '不会',
 2070: '因为',
 4750: '考虑',
 2927: '性能',
 2607: '屏蔽掉',
 3781: '核心',
 4353: '相反',
 1005: '他们',
 4776: '联手',
 2889: '微软',
 3111: '找到',
 466: '一种',
 5424: '适合',
 1376: '兼顾

In [85]:
id_2_word[5600]

'阶段'

In [88]:
set(np.where(transposed_x[5600])[0]) # 每个content是否包含的‘阶段’词

{7, 19, 26, 47}

In [89]:
from functools import reduce # 累积计算
from operator import and_ # 按位与 
# "按位与"
# 1&1=1
# 1&0=0
# 0&1=0
# 0&0=0

d1, d2, d3 = {1, 2, 3}, {4, 5, 6, 3, 2}, {1, 3, 4}
reduce(and_, [d1, d2, d3])

{3}